In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os.path as osp
import pathlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn import Sequential, ReLU, Linear, Dropout, BatchNorm1d
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import weight_norm
import networkx as nx

import optuna

# from importlib import reload  
# import src.models
# reload(src.models)
from src.models import TrEnc, GraphNet, Vects2Edge, Vects2Choice, GraphAgg,add_weight_decay
from src.models import FeatureTransform, ClassDiff, NodeExpand
from src.radam import RAdam
from src.data_loading import data_gen_edges, get_cora, get_dataset
from src.utils import create_nx_graph
from src.dev import data_gen_edges2, train, test
# calc_BA, 
from src.train import LabelSmoothing, cal_edge_loss

from collections import deque
from torch_geometric.utils import k_hop_subgraph


import numpy as np
import torch
import networkx as nx


In [13]:
B_SIZE = 16


In [16]:
# d_train, d_test, _, data = get_dataset('full', 'cora', '/data/egor/graph_generation/graph_generation/data')
d_train, d_test, _, data0 = get_dataset('plan', 'Cora', '/data/egor/graph_generation/graph_generation/data')

data = tt_split_edges(data0)


In [2]:
def tt_split_edges(dat):
    torch.manual_seed(0)
    train_test_ratio = 0.2

    row, col = dat.edge_index
    mask = row < col
    row, col = row[mask], col[mask]

    e_mask = torch.FloatTensor(row.shape[0]).uniform_() >train_test_ratio
    dat.train_test_mask = torch.cat((e_mask, e_mask))
    dat.edge_index = torch.cat( (torch.stack((row, col ), dim=1).T,
                              torch.stack((col,row), dim=1).T), dim=1)
    return dat

In [19]:
tt_split_edges(data0)

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], train_test_mask=[10556], val_mask=[2708], x=[2708, 1433], y=[2708])

In [3]:
data, data0 = get_cora()
data.x = data.x.to(dtype=torch.float)
data = tt_split_edges(data)

In [64]:
def calc_BA(grp, n_nodes):
    v,  cnt = torch.unique(grp, return_counts=True)
    zrs = torch.zeros(n_nodes)
    zrs[v] = cnt.to(dtype=torch.float)/2
    return zrs

def data_gen_edges_dyn(data, batch_size = 4, stepf = 8):
    mask = data.edge_index[0] < data.edge_index[1]
    n_edges = int(mask.sum().item())
    edge_perm = []
    for i in range(batch_size):
#         edge_index.append(data.edge_index[:,mask][:,torch.randperm(n_edges)] + data.num_nodes *i)
        edge_perm.append(torch.randperm(n_edges))
    edge_perm = torch.stack(edge_perm)   
    

    n_steps = batch_size
    step_max = 64
    ratio = 0.1

    n_min = 64
    ind_array = (np.linspace((n_min+step_max)/n_edges,1,n_steps)*n_edges).astype(int)
    ff = data.num_nodes *torch.stack((torch.arange(16),torch.arange(16))).unsqueeze(2)
#     tt_mask = data.train_test_mask[edge_perm]
    
    for i in ind_array:
        
        dd = data.edge_index[:,mask][:,edge_perm]+ff
        
        
#         graph = edge_index[:,:,:(i-step_max)]
#         e_1 = edge_index[:,:,(i-step_max):i].reshape((2,-1))
#         BA_factors = calc_BA(graph, data.num_nodes * batch_size).view((batch_size,data.num_nodes))
        
        
        e_1_inds = edge_perm[:,(i-step_max):i]
        e_1 = (data.edge_index[:, mask][:,e_1_inds]+ff).reshape((2,-1))
        
        tt_mask = data.train_test_mask[e_1_inds].reshape((-1,))
        print(e_1[:,tt_mask].shape)
        return 1
        
    
    
    
        ba_sm = BA_factors.sum(dim=1)
#         print(ba_sm.shape, BA_factors.shape)
        BA_factors = BA_factors/ba_sm.unsqueeze(1)
        # *********  negative edges 
        r1,c1 = e_1
        c0 = torch.zeros(size=c1.shape)
        c0[0] = c1[ -1] # hz, rabotaet li????????????????
        c0[ 1:] = c1[:-1] # smeshaem po krugu
        e_0 = torch.stack( (r1,c0))
                    
        vmask = torch.full((data.num_nodes*batch_size,), False,  dtype=torch.bool)
        vmask[torch.unique(graph)] = True             
        yield graph.reshape((2,-1)) , torch.cat((e_0,e_1), dim=1).to(dtype = torch.long), BA_factors.view(-1,), vmask

In [65]:
for inp in data_gen_edges_dyn(data, batch_size = 16, stepf = 8):
    pass
#     gr, ed, ba, v = inp
#     print(ba.shape)

torch.Size([2, 812])


In [9]:
def create_model(num_features, arch, device):
    dc = ModelFull2(num_features, device, arch)
    dc = dc.to(device = device)
    prs = add_weight_decay(dc.named_parameters(), arch['weight_decay'])
    optimizer = RAdam(prs, lr=arch['lr'])
    return dc, optimizer


class ModelFull2(torch.nn.Module):
    def __init__(self, num_features, device, arch):
        super().__init__()
        
        enc_layers = []
        enc_layers.append(FeatureTransform(num_features, arch['nd']))
        if arch['add_trans']:
            enc_layers.append(TrEnc(arch['nd'], arch['nd']))
        self.encoder = nn.Sequential(*enc_layers)
        self.graph_net = GraphNet(arch)        
        self.v2_edge = ClassDiff(arch['nd'], arch['fin_type_e'], 0)
#         self.gagg = GraphAgg(arch['nd'], arch['agg_type'])     
        self.gru = nn.GRUCell(arch['nd']*2, arch['nd'])

        self.device = device
        self.nd = arch['nd']
        self.ls = LabelSmoothing()
        self.label_smooth = arch['lb_smth']
                
    @torch.no_grad()    
    def reset_state(self, num_nodes):
        self.state = torch.zeros((num_nodes,  self.nd), device=self.device)
        
        
    def iterate(self, data, inp, b_size): 
        print(data)
        gr, edges, BA_fac, v_in_gr = inp
        BA_fac = BA_fac.to(device = self.device)
        gr = gr.to(device = self.device)
        v_in_gr = v_in_gr.to(device = self.device)
        edges = edges.to(device = self.device)
        encoded = self.encoder(data.x.to(device = self.device))
        
        decoded =  self.decode(gr, encoded, b_size, data.num_nodes)        
        
        e_v_1, e_v_2 = edges    
        
        pred_e = self.v2_edge(self.state[e_v_1], self.state[e_v_2]) 
        
#         pred_e = self.v2_edge(decoded[e_v_1], decoded[e_v_2])   
#         zrs = torch.zeros(edges.shape[1],1, device=self.device)
#         pred_e = self.v2_edge(torch.cat((decoded[e_v_1],zrs, BA_fac[e_v_1].unsqueeze(1)), dim=1), 
#                               torch.cat((decoded[e_v_2],BA_fac[e_v_2].unsqueeze(1), zrs), dim=1))

        n_e = edges.shape[1]//2
        targ_e = torch.cat((torch.zeros(n_e, device =  self.device),
                            torch.ones(n_e, device =  self.device)))
                
        if self.label_smooth:
            loss_e = cal_edge_loss(pred_e, targ_e)
        else:
            loss_e = F.binary_cross_entropy(pred_e, targ_e)    

        loss = loss_e 
        return  pred_e, targ_e, loss
    
    def encode(self, xx):
        return self.encoder(xx.to(device = self.device))
        
    def decode(self, gr, x_enc, b_size, num_nodes = None):    
        if num_nodes is not None:
            indd = torch.cat(b_size*[torch.arange(num_nodes, device = self.device)])
            decoded = self.graph_net( x=x_enc[indd,:], edge_index = gr)
            self.state = self.gru(torch.cat((x_enc[indd,:], decoded),dim=1), self.state)
        else:
            decoded = self.graph_net( x=x_enc, edge_index = gr)   
            self.state = self.gru(torch.cat((x_enc, decoded),dim=1), self.state)
        return decoded

In [10]:
# device = torch.device('cpu')
device = torch.device('cuda:1')
arch0 = {'nd': 64, 'weight_decay': 0.0007,'lr': 0.0001,'nlayers_gnn': 3,'add_trans': False, 'conv_type':'gin',
        'node_agg_type': 'max','agg_type': 'sum_gate','fin_type_e': 'diff_mlp',
        'lb_smth': True}
# diff_mlp
# het
model, optimizer = create_model(data.num_features, arch0, device)

In [11]:
# G_test, labs_test = create_nx_graph(d_test.edge_index, d_test.y, get_largest = False)
# plt.figure(figsize = (100, 100))
# nx.draw(G_test, node_color=labs_test, node_size =2000)

In [14]:
num_epochs = 2
for epoch in range(1, num_epochs + 1):
#     d_train
    train(data_gen_edges_dyn, model, optimizer, data, B_SIZE)
#     train(data_gen_edges_dyn, model, optimizer, d_train, B_SIZE)
#     train(data_gen_edges_dyn, model, optimizer, data, B_SIZE)

#     acc_e_test = test( data_gen_edges_dyn, model, d_test, 1)
    acc_e_test = test( data_gen_edges_dyn, model, data, 1)

    print(acc_e_test.mean())

Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])


/data/anaconda3/envs/genv/lib/python3.8/site-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
/data/egor/graph_generation/graph_generation/src/radam.py:58: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370117127/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])
Data(edge_index=[2, 10138], train_test_mask=[10138], x=[2485, 1433], y=[2485])


KeyboardInterrupt: 

In [8]:
device=torch.device('cpu')
model = model.to(device = device)
model.device = device

In [26]:
def sel_start_node(nds):
    _, degs = torch.unique(data.edge_index, return_counts = True)
    deginds = torch.argsort(degs[nds], descending=True)
    sel_ind = nds[deginds[deginds.shape[0]//2]]
    return sel_ind

@torch.no_grad()
def generate_graph(Nnodes = 500):
    def add_nodes(gr):
        decoded =  model.decode(gr, encoded, 1)  
        n_edges = gr.shape[1]
        decoded = model.graph_net( x=encoded, edge_index = gr)
        
        
        zrs = torch.zeros(decoded.shape[0],1, device=device)
        ba_f = calc_BA(gr, decoded.shape[0]).unsqueeze(1)
#         dec = model.v2_edge(decoded[None,:,:],decoded[:,None, :])
        dec = model.v2_edge(torch.cat((decoded,zrs, ba_f),dim=1)[None,:,:], 
                            torch.cat((decoded,ba_f, zrs),dim=1)[:,None, :])
        decision = torch.triu( dec, diagonal=1)
        
        c,r = gr
        decision[c,r] = 0
        decision[r,c] = 0
        
        n_new = int(n_edges * 0.1)
        ff = torch.argsort(torch.flatten(decision),  descending=True)
        innd = ff[n_new]
        thr = max(torch.flatten(decision)[innd], 0.5)
#         print(thr)
#         thr = 0.5
        res = (decision >=thr).nonzero(as_tuple=False).T[:, :n_new]
#         print(res)
        return torch.cat((res, torch.flip(res, dims=(0,))), dim=1)
    
    model.eval()
    node_set = torch.randperm(data.num_nodes)[:Nnodes].to(device = device)
    start_nds, ei0, _, _ = k_hop_subgraph([sel_start_node(node_set)], 2, data.edge_index,relabel_nodes=True)
    
    node_set = torch.cat((node_set, start_nds.to(device = device)))
    X = data.x[node_set,:]
    Y = data.y[node_set]
    
    encoded = model.encode(X.to(device = device))
    gr = ei0.to(device = device)
    model.reset_state(node_set.shape[0])
   
    while gr.shape[1]< Nnodes*2:
#     for _ in range(50):
        new_nodes = add_nodes(gr)
        gr = torch.cat((gr, new_nodes), dim=1)
#         if gr.shape[1]> Nnodes*7:
#             break
        print(gr.shape)
        
    return gr, Y

In [27]:
grr, lbs = generate_graph(Nnodes = 1000)

torch.Size([2, 444])
torch.Size([2, 532])
torch.Size([2, 638])
torch.Size([2, 764])
torch.Size([2, 916])
torch.Size([2, 1098])
torch.Size([2, 1316])
torch.Size([2, 1578])
torch.Size([2, 1892])
torch.Size([2, 2270])


In [28]:
G, labs2 = create_nx_graph(grr, lbs, get_largest = False)
plt.figure(figsize = (100, 100))
nx.draw(G, node_color=labs2, node_size =2000)

In [19]:
# G, labs2 = create_nx_graph(grr, lbs, get_largest = False)
# plt.figure(figsize = (100, 100))
# nx.draw(G, node_color=labs2, node_size =2000)

In [29]:
nx.average_clustering(G)

0.38191912298896535

In [24]:
G.number_of_nodes()

555

In [25]:
G.number_of_edges()

2761

In [ ]:
G.number_of_nodes()

In [68]:
inp = torch.randn(1, 100, hidden_size)
h0 = torch.randn(1, 100, hidden_size)

In [69]:
output, hn = gru(inp, h0)

In [ ]:
def data_gen_edges_dyn(data, batch_size = 4, stepf = 8):
    mask = data.edge_index[0] < data.edge_index[1]
    n_edges = int(mask.sum().item())
    edge_index = []
    for i in range(batch_size):
        edge_index.append(data.edge_index[:,mask][:,torch.randperm(n_edges)] + data.num_nodes *i)
    edge_index = torch.stack(edge_index)       
    edge_index =  torch.transpose(edge_index, 0, 1)

    n_steps = batch_size
    step_max = 64
    ratio = 0.1

    n_min = 64
    ind_array = (np.linspace((n_min+step_max)/n_edges,1,n_steps)*n_edges).astype(int)

    for i in ind_array:
        graph = edge_index[:,:,:(i-step_max)]
        e_1 = edge_index[:,:,(i-step_max):i].reshape((2,-1))
        BA_factors = calc_BA(graph, data.num_nodes * batch_size).view((batch_size,data.num_nodes))
        
        ba_sm = BA_factors.sum(dim=1)
#         print(ba_sm.shape, BA_factors.shape)
        BA_factors = BA_factors/ba_sm.unsqueeze(1)
        # *********  negative edges 
        r1,c1 = e_1
        c0 = torch.zeros(size=c1.shape)
        c0[0] = c1[ -1] # hz, rabotaet li????????????????
        c0[ 1:] = c1[:-1] # smeshaem po krugu
        e_0 = torch.stack( (r1,c0))
                    
        vmask = torch.full((data.num_nodes*batch_size,), False,  dtype=torch.bool)
        vmask[torch.unique(graph)] = True             
        yield graph.reshape((2,-1)) , torch.cat((e_0,e_1), dim=1).to(dtype = torch.long), BA_factors.view(-1,), vmask

In [ ]:
def custom_loss_function(x, labels):
    y = F.cross_entropy(x, labels[:, 0], reduction="none")
    y = th.log(epsilon + y) - math.log(epsilon)
    return th.mean(y)




#TOPOLOSS   !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # randomly sample edges to calculate topology consistency
    mask = torch.randn(graph.num_edges(), device=pred.device) < args.topo_mask_threshold
    srcs = torch.masked_select(graph.edges()[0], mask)
    dsts = torch.masked_select(graph.edges()[1], mask)
    pred_srcs = pred[srcs]
    pred_dsts = pred[dsts]
    topo_consistency = torch.nn.functional.cosine_similarity(pred_srcs, pred_dsts).mean()
    topo_loss = 1 - topo_consistency

    if n_label_iters > 0:
        unlabel_idx = torch.cat([train_pred_idx, val_idx, test_idx])
        for _ in range(n_label_iters):
            feat[unlabel_idx, -n_classes:] = F.softmax(pred[unlabel_idx].detach(), dim=-1)
            pred = model(graph, feat)

    ce_loss = custom_loss_function(pred[train_pred_idx], labels[train_pred_idx])
    loss = ce_loss + args.topo_loss_ratio * topo_loss

    loss.backward()
    optimizer.step()

    return ce_loss.item(), pred.detach(), topo_loss.item()

In [38]:
nx.average_clustering(G)

0.40109011307345543

In [25]:
nx.average_clustering(G0)

NameError: name 'G0' is not defined

In [ ]:
# G0, labs0 = create_nx_graph(data.edge_index, data.y)
# plt.figure(figsize = (100, 100))
# nx.draw(G0, node_color=labs0, node_size =2000)

In [ ]:
for inp in data_gen(data, adjacency, start_points, shuffle=False):
    inpp = inp
    break

with torch.no_grad():
    for w0 in np.linspace(-10,10,51):
        models['v2_choice'].lin0_vc.weight.data.fill_(w0)
        models['v2_edge'].lin0_ve.weight.data.fill_(w0)
        tp_v, n_v, pred_e, targ_e, n_e_sum, _, l_e,l_c = iterate(models, inpp, len(start_points))
        print('{:.2f}'.format(w0), l_e.item(),l_c.item())